In [1]:
%matplotlib inline
import os
import h5py
import matplotlib.pyplot as plt
from ml4cvd.arguments import _get_tmap
from ml4cvd.tensor_generators import get_test_train_valid_paths
from ml4cvd.tensor_from_file import _decompress_data
import numpy as np
import numcodecs
import glob

/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '
Using TensorFlow backend.


In [2]:
#tm_rate = _get_tmap('partners_ecg_rate')
#tm_qrs = _get_tmap('partners_ecg_qrs')
#tm_pr = _get_tmap('partners_ecg_pr')
tm_qt = _get_tmap('partners_ecg_qt')
tm_qt_test = _get_tmap('partners_ecg_qt_test')
#tm_qtc = _get_tmap('partners_ecg_qtc')
#tm_read = _get_tmap('supranodal_rhythms')
# tm_voltage = _get_tmap('partners_ecg_voltage_raw')

In [6]:
%matplotlib inline

fpaths = glob.glob("/home/erik/partners_ecg/hd5/2019-01/*hd5")
qt = np.zeros(len(fpaths))
qt_test = np.zeros(len(fpaths))

for i, f in enumerate(fpaths):
    with h5py.File(f, 'r') as hd5:
        #rate = tm_rate.tensor_from_file(tm_rate, hd5)
        #qrs = tm_qrs.tensor_from_file(tm_qrs, hd5)
        #pr = tm_pr.tensor_from_file(tm_pr, hd5)
        try:
            qt[i] = tm_qt.tensor_from_file(tm_qt, hd5)
            qt_test[i] = tm_qt_test.tensor_from_file(tm_qt_test, hd5)
        except:
            continue
        # qtc = tm_qtc.tensor_from_file(tm_qtc, hd5)
        # read = tm_read.tensor_from_file(tm_read, hd5)
        # print(f"rate: {rate}, qrs: {qrs}, pr: {pr}, qt: {qt}, qtc: {qtc}, read: {read}")
        #print(f"qt: {qt}, qt_test: {qt_test}")

In [7]:
df = {'qt': qt, 'qt_test': qt_test}

In [8]:
import pandas as pd
df = pd.DataFrame(df)

In [13]:
df[df['qt'] < 1.0]

,qt,qt_test
1226,0.0,0.0
6042,0.0,0.0
8311,0.0,0.0
12341,0.0,0.0
